In [ ]:
import sympy
import numpy as np
import matplotlib.pyplot as plt
from sympy import lambdify
import dill
dill.settings["recurse"] = True
from tqdm.notebook import tqdm
import plotly
from triangle import triangulate

In [ ]:
weak_form_functional_lambdified = dill.load(open("../calculations/nonconforming_linear_crouzeix_raviart_weak_form_functional", "rb"))
weak_form_right_part_lambdified = dill.load(open("../calculations/nonconforming_linear_crouzeix_raviart_weak_form_right_part", "rb"))
interpolation_function = dill.load(open("../calculations/nonconforming_linear_crouzeix_raviart_basis", "rb"))
mapping_function_lambdified = dill.load(open("../calculations/nonconforming_linear_crouzeix_raviart_mapping_function", "rb"))

In [ ]:
t = np.linspace(0, 2 * np.pi, 100, endpoint=False)

points = np.stack((np.cos(t), np.sin(t))).T

seg = np.array(
    [[j, j + 1] for j in range(points.shape[0] - 1)] + [[points.shape[0] - 1, 0]]
)

In [ ]:
cndt = triangulate({"vertices": points, "segments": seg}, opts="pq30a0.1neo1s")
points = cndt["vertices"]
triangles = cndt["triangles"]
neighbours = cndt["neighbors"]
vertex_marker_is_boundary = cndt["vertex_markers"]
edges = cndt["edges"]
edge_marker_is_boundary = cndt["edge_markers"]

print(f"Total number of points {points.shape[0]}")

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7, 7))
ax.axis("equal")

cl = ["black", "brown"]

for eidx, edge in enumerate(edges):
    ax.plot(
        points[edge][:, 0],
        points[edge][:, 1],
        color=cl[edge_marker_is_boundary[eidx][0]],
    )

ax.scatter(*points.T, c=vertex_marker_is_boundary)

ax.set_xlim([points[:, 0].min() - 1 / 2, points[:, 0].max() + 1 / 2])
ax.set_ylim([points[:, 1].min() - 1 / 2, points[:, 1].max() + 1 / 2])

fig.tight_layout()
plt.tight_layout()

plt.show()

In [ ]:
x, y = sympy.symbols("x y")

In [ ]:
F = sympy.Float(1)
right_vals = np.zeros((edges.shape[0]))
right_vals[:] = sympy.lambdify((x, y), F)(*points[edges].mean(axis=-1).T)

In [ ]:
def get_local_edges(points, triangle, sorted_edges):
    result = []

    for pair in triangle[np.array([[1, 2], [0, 2], [0, 1]])]:
        pairs_sorted_idx = np.argsort(pair)
        idx = np.where(np.all(pair[pairs_sorted_idx] == sorted_edges, axis=-1))[0][
            0
        ]
        result.append(idx)

    return result


triangle_to_edge = np.zeros_like(triangles)
edges_sorted_idx = np.argsort(edges, axis=-1)
sorted_edges = np.take_along_axis(edges, edges_sorted_idx, axis=-1)

# associate edges with triangles
for tidx, triangle in enumerate(triangles):
    result = get_local_edges(points, triangle, sorted_edges)
    triangle_to_edge[tidx] = result
    
triangle_to_edge.sort(axis=-1)

In [ ]:
edge_middpoints = points[edges].mean(axis=-2)

In [ ]:
matrix = np.zeros((edges.shape[0], edges.shape[0]))
b = np.zeros(edges.shape[0])

for tidx, element in enumerate(tqdm(triangle_to_edge)):
    
    x1, x2, x3 = edge_middpoints[element[0], 0], edge_middpoints[element[1], 0], edge_middpoints[element[2], 0]
    y1, y2, y3 = edge_middpoints[element[0], 1], edge_middpoints[element[1], 1], edge_middpoints[element[2], 1]
    
    wff = weak_form_functional_lambdified(x1, y1, x2, y2, x3, y3)
    wfrp = weak_form_right_part_lambdified(
                                            x1, y1, x2, y2, x3, y3,
                                            right_vals[element[0]], right_vals[element[1]], right_vals[element[2]]
                                           )
    
    for local_edge in range(3):
        
        if edge_marker_is_boundary[element[local_edge]] == True:
            
            matrix[element[local_edge], element[local_edge]] = 1
            b[element[local_edge]] = 0                                            
        else:
            for dofidx in range(3):
                
                matrix[element[local_edge], element[dofidx]] += 2 * wff[local_edge, dofidx]                
            b[element[local_edge]] -= 2 * wfrp[local_edge]

In [ ]:
F = np.linalg.solve(matrix, b)

In [ ]:
param = np.linspace(0, 1, 10)
gridx, gridy = np.meshgrid(param, param, indexing="ij")

mask = (gridy <= 1 - gridx)

gridx = gridx[mask]
gridy = gridy[mask]

In [ ]:
global_x, global_y, global_z = [], [], []

for element in triangle_to_edge:
    
    x1, x2, x3 = edge_middpoints[element[0], 0], edge_middpoints[element[1], 0], edge_middpoints[element[2], 0]
    y1, y2, y3 = edge_middpoints[element[0], 1], edge_middpoints[element[1], 1], edge_middpoints[element[2], 1]
    
    physical_x, physical_y = mapping_function_lambdified(x1, y1, x2, y2, x3, y3, gridx, gridy)[:, 0]
    
    physical_z = interpolation_function(
                                        x1, y1, x2, y2, x3, y3,
                                        F[element[0]], F[element[1]], F[element[2]],
                                        gridx, gridy
                                        )
    
    global_x.append(physical_x)
    global_y.append(physical_y)
    global_z.append(physical_z)
    
# global_x = np.concatenate(global_x)
# global_y = np.concatenate(global_y)
# global_z = np.concatenate(global_z)

In [ ]:
# plot_data = [
#     plotly.graph_objects.Mesh3d(
#                                 x=global_x, 
#                                 y=global_y,
#                                 z=global_z,
#                                 intensity=global_z
#                                 )        
#     ]
# fig = plotly.graph_objects.Figure(data=plot_data)
# fig.update_layout()

In [ ]:
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=gx, 
                                y=gy,
                                z=gz, 
                                )
        for gx, gy, gz in zip(global_x, global_y, global_z)
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()

In [ ]:
true_solution = (global_x**2 + global_y**2 - 1)/4
(global_z - true_solution).max()